In [ ]:
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
OPENAI_API_KEY = " your api key here " 

In [ ]:
from langchain.llms import openai

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

In [ ]:
llm.invoke("Explain EDA in 2 lines")

In [ ]:
#using custom dataset

pdf_reader = PyPDFLoader("/content/RAG_Paper.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_document(documents)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
db = FAISS.from_documents(documents = chunk , embedding = embeddings)


In [ ]:
from langchain.chains import ConversationalRetreivalChain
from langchain.prompts import PromptTemplate

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template
                           (""" Given the following conversation and a follow 
                                 up question , rephrase the follow-up 
                                 question to be a standalone question")

                                 Chat History :
                                         ( {chat_history}
                                          follow up input = {question}
                                          standalone question """ )


In [ ]:
qa = ConversationalRetreivalChain.from_llm(llm=llm,
            retreiver = db.as_retreiver(),
            condense_question_prompt = CONDENSE_QUESTION_PROMPT,
            return source_documents=True,
            verbose = False)

In [ ]:
chat_history=[]
query = """ What is this research paper all about """
result = qa ({"question" : query , "chat_history" : chat_history})
print(result["answer])